In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pygame
import sys
import cv2
import pickle
from keras.models import load_model

pygame 2.5.2 (SDL 2.28.2, Python 3.9.18)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-04-05 14:25:58.516245: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-05 14:25:58.517525: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 14:25:58.541272: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-05 14:25:58.541894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 14:25:59.017840: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

In [2]:
# Initialize Pygame
pygame.init()

# Model stuff 
with open('encoder.pickle', 'rb') as f:
    # Use pickle.load() to load the data
    encode = pickle.load(f)

model = load_model("my_model_81_accuracy.h5")

main_count = 0
# Set the width and height of the screen (adjust as needed)
SCREEN_WIDTH = 320
SCREEN_HEIGHT = 320

# Set up the screen
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
pygame.display.set_caption("Drawing with Pygame")

# Set colors
white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)
green = (0,255,0)

# Set circle parameters
circle_radius = 10  # Adjust as needed

# Open camera
cap = cv2.VideoCapture(0)  # Use 0 for the default camera, change if you have multiple cameras

# Set font
font = pygame.font.Font(None, 36)

# set medssage
pygame.display.set_caption("Text Display Example")

def main_message():
    # Render text onto a surface
    text_surface = font.render('Hello, Lets get drawing!\n press Enter to begin.', True, red)  # Change the text as needed

    # Blit the text surface onto the screen
    screen.blit(text_surface, (SCREEN_WIDTH // 2 - text_surface.get_width() // 2, SCREEN_HEIGHT // 2 - text_surface.get_height() // 2))

def predict(animal):
    # Render text onto a surface
    text_surface = font.render(f'{animal}', True, green)  # Change the text as needed
    # Blit the text surface onto the screen
    screen.blit(text_surface, (SCREEN_WIDTH // 2 - text_surface.get_width() // 2, SCREEN_HEIGHT // 2 - text_surface.get_height() // 2))

def preprocessing():
    image = cv2.imread("snapshot.jpg")
    image = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    image = cv2.resize(image, (28, 28))
    image = np.reshape(image, (28,28,1))
    input_image = np.expand_dims(image, axis=0)  # Add batch dimension
    prediction = model.predict(input_image)
    max_index = np.argmax(prediction)
    one_hot_encoded = np.zeros_like(prediction)
    one_hot_encoded[0][max_index] = 1
    animal = encode.inverse_transform(np.reshape(one_hot_encoded,(1,-1)))[0][0]
    print(f"prediction is {animal}")
    return animal

while(cap.isOpened()):
    
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame vertically
    frame = cv2.flip(frame, 0)

    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Thresholding to isolate bright areas (IR light)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Iterate through contours
    for contour in contours:
        # Compute centroid of contour
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])

            # Draw centroid on the frame
            cv2.circle(frame, (cX, cY), 5, (0, 0, 255), -1)
            # Print normalized coordinates
            video_height, video_width, _ = frame.shape
            normalized_cX = cX / video_width
            normalized_cY = cY / video_height

            # # Display frame
            # cv2.imshow('IR Detection', frame)
            
            # Draw circle at the centroid in Pygame
            pygame.draw.circle(screen, white, (int(cX * SCREEN_WIDTH / frame.shape[1]), int(cY * SCREEN_HEIGHT / frame.shape[0])), circle_radius)
    if main_count == 0:
        main_message()
    
    # Update the display
    pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_q: 
                cap.release()
                cv2.destroyAllWindows()
                pygame.quit()
                # sys.exit()
            elif event.key == pygame.K_c:
                screen.fill(black)
            elif event.key == pygame.K_RETURN:
                main_count += 1
                if main_count != 0:
                    screen.fill(black)
            elif event.key == pygame.K_s:
                pygame.image.save(screen, "snapshot.jpg")
                animal = preprocessing()
                screen.fill(black)
                predict(animal)

2024-04-05 14:25:59.868047: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-05 14:25:59.868391: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 [==============================] - 0s 98ms/step
prediction is hedgehog
